# LAMMPS Tutorials 01. Running your first LAMMPS simulation!

### Author: Mark Tschopp, mark.a.tschopp.civ at mail.mil

Please contact me if you have a problem with this tutorial, so I can modify in Github.  I have added FAQs, and will update my versions of LAMMPS in the future to keep the scripts current.

The latest version of this [Jupyter Notebook](http://ipython.org/notebook.html) tutorial is available at https://github.com/mrkllntschpp/lammps-tutorials.

The original tutorials are given here: https://icme.hpc.msstate.edu/mediawiki/index.php/LAMMPS_tutorials.  A number of these tutorials are out of date and have been ported over into the current iPython Jupyter Notebook tutorials on github.

***

## Description:
<a id="Sec1"></a>
This is a quick tutorial to running a LAMMPS simulation on a Windows machine. For this simple example, the molecular simulation calculates the equilibrium lattice constant and corresponding cohesive energy for aluminum in the face-centered cubic (fcc) lattice structure (see below).

<img src="https://icme.hpc.msstate.edu/mediawiki/images/e/ef/Fcc_stereo.gif" alt="Face-centered Cubic Lattice Structure" title="FCC Lattice Structure" />

***
## Step 1: Download an Input File
<a id="Step1"></a>

This input script was run using the Aug 2015 version of LAMMPS. Changes in some commands may require revision of the input script. To get the input file, you have a few options:

*  Copy the text below and paste it into a text file, `calc_fcc.in`. Use the `Paste Special` command with unformatted text into the file. 
*  Or, I added the command `%%writefile calc_fcc.in` to the Jupyter Notebook which should just do everything for you!

Obviously, if the second option worked, you are all set!

In [1]:
%%writefile calc_fcc.in
######################################
# LAMMPS INPUT SCRIPT
# Find minimum energy fcc (face-centered cubic) atomic configuration
# Mark Tschopp
# Syntax, lmp_exe < calc_fcc.in

######################################
# INITIALIZATION
clear 
units metal 
dimension 3 
boundary p p p 
atom_style atomic 
atom_modify map array

######################################
# ATOM DEFINITION
lattice fcc 4 orient x 1 1 0 orient y -1 1 0 orient z 0 0 1  
region box block 0 1 0 1 0 1 units lattice
create_box 1 box
create_atoms 1 box
replicate 1 1 1

######################################
# DEFINE INTERATOMIC POTENTIAL
pair_style eam/alloy 
pair_coeff * * Al99.eam.alloy Al
neighbor 2.0 bin 
neigh_modify delay 10 check yes 
 
######################################
# DEFINE COMPUTES 
compute eng all pe/atom 
compute eatoms all reduce sum c_eng 

#####################################################
# MINIMIZATION
reset_timestep 0 
fix 1 all box/relax iso 0.0 vmax 0.001
thermo 10 
thermo_style custom step pe lx ly lz press c_eatoms 
min_style cg 
minimize 1e-25 1e-25 5000 10000 

variable natoms equal "count(all)" 
variable teng equal "c_eatoms"
variable length equal "lx"
variable ecoh equal "v_teng/v_natoms"

print "Total energy (eV) = ${teng};"
print "Number of atoms = ${natoms};"
print "Lattice constant (Angstoms) = ${length};"
print "Cohesive energy (eV) = ${ecoh};"

print "All done!"

Writing calc_fcc.in


Awesome!  That little script should have written the above text to the file `calc_fcc.in`.  To check, let's execute a command on the present directory listing all files that end in `*.in`.

In [2]:
!dir *.in

calc_fcc.in


***
## Step 2: Running LAMMPS
### Run this using LAMMPS in Jupyter Notebook
We can actually run this from Jupyter Notebook.  Let's try it.

In [3]:
!lmp_serial < calc_fcc.in

LAMMPS (3 Mar 2020)
Lattice spacing in x,y,z = 5.65685 5.65685 4
Created orthogonal box = (0 0 0) to (5.65685 5.65685 4)
  1 by 1 by 1 MPI processor grid
Created 8 atoms
  create_atoms CPU = 0.000447035 secs
Replicating atoms ...
  orthogonal box = (0 0 0) to (5.65685 5.65685 4)
  1 by 1 by 1 MPI processor grid
  8 atoms
  replicate CPU = 0.000544786 secs
Neighbor list info ...
  update every 1 steps, delay 0 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 8.28721
  ghost atom cutoff = 8.28721
  binsize = 4.1436, bins = 2 2 1
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair eam/alloy, perpetual
      attributes: half, newton on
      pair build: half/bin/atomonly/newton
      stencil: half/bin/3d/newton
      bin: standard
Setting up cg style minimization ...
  Unit style    : metal
  Current step  : 0
Per MPI rank memory allocation (min/avg/max) = 4.474 | 4.474 | 4.474 Mbytes
Step PotEng Lx Ly Lz Press c_eatoms 
       0 

Sweet!

If you want to view the entire file (opening `log.lammps` in Notepad), then:

In [4]:
!cat log.lammps

LAMMPS (3 Mar 2020)
######################################
# LAMMPS INPUT SCRIPT
# Find minimum energy fcc (face-centered cubic) atomic configuration
# Mark Tschopp
# Syntax, lmp_exe < calc_fcc.in

######################################
# INITIALIZATION
clear
units metal
dimension 3
boundary p p p
atom_style atomic
atom_modify map array

######################################
# ATOM DEFINITION
lattice fcc 4 orient x 1 1 0 orient y -1 1 0 orient z 0 0 1
Lattice spacing in x,y,z = 5.65685 5.65685 4
region box block 0 1 0 1 0 1 units lattice
create_box 1 box
Created orthogonal box = (0 0 0) to (5.65685 5.65685 4)
  1 by 1 by 1 MPI processor grid
create_atoms 1 box
Created 8 atoms
  create_atoms CPU = 0.000447035 secs
replicate 1 1 1
  orthogonal box = (0 0 0) to (5.65685 5.65685 4)
  1 by 1 by 1 MPI processor grid
  8 atoms
  replicate CPU = 0.000544786 secs

######################################
# DEFINE INTERATOMIC POTENTIAL
pair_style eam/alloy
pair_coeff * * Al99.eam.alloy Al
neighbo

***
## LAMMPS Input Script Explained

In this section, we will break down what LAMMPS is doing for each step. The easy way to do this on your own is to consult the LAMMPS manual for each command or go to the Internet LAMMPS manual, *i.e.*, at https://lammps.sandia.gov

<br>
<div class="alert alert-block alert-info">
    # Find minimum energy fcc configuration  <br>
# Mark Tschopp </div>

The `#` denotes a comment. Hence LAMMPS does not do anything for these lines. 

<br>
<div class="alert alert-block alert-info">
# ---------- Initialize Simulation ---------------------  <br>
clear  <br>
units metal  <br>
dimension 3  <br>
boundary p p p  <br>
atom_style atomic  <br>
atom_modify map array <br>
</div>

This section initializes the simulations. The `clear` command clears all memory. The `units` command specifies the units that will be used for the remainder of the simulation; `metal` uses Angstroms and eV, among other units. The `dimension 3` command specifies a 3-dimensional simulation cell will be used (2-D is also an option). The `boundary p p p` specifies periodic boundaries in the x-, y-, and z-directions. 

<br>
<div class="alert alert-block alert-info">
# ---------- Create Atoms ---------------------  <br>
lattice 	fcc 4 <br>
region	box block 0 1 0 1 0 1 units lattice <br>
create_box	1 box <br> <br>
lattice	fcc 4 orient x 1 0 0 orient y 0 1 0 orient z 0 0 1   <br>
create_atoms 1 box <br>
replicate 1 1 1 <br></div>

The `lattice` command specifies what type of lattice is used (fcc, bcc, hcp, etc.) and the number following this specifies the lattice constant. The `region` command specifies the simulation cell. Here, we have used lattice units and specified that the simulation cell box is to be 1 lattice unit in each direction. The `create_box` command following this will use the parameters outlined in the `region` command to actually create the box. The `replicate` command can be used to replicate the periodic cell in each direction. 

<br>
<div class="alert alert-block alert-info">
# ---------- Define Interatomic Potential --------------------- <br>
pair_style eam/alloy  <br>
pair_coeff * * Al99.eam.alloy Al <br>
neighbor 2.0 bin  <br>
neigh_modify delay 10 check yes  <br>
</div>

The `pair_style` command specifies what kind of interatomic potential will be used, while the `pair_coeff` specifies the file that the pair potential coefficients are stored in. The file extension for the interatomic potential file can sometimes give a clue as to which format is being used (eam.alloy = eam/alloy).

<br>
<div class="alert alert-block alert-info">
# ---------- Define Settings --------------------- <br>
compute eng all pe/atom <br>
compute eatoms all reduce sum c_eng <br>
</div>

Here, two computes are defined. In the first `compute` command, a variable named `eng` is defined to store the potential energy for each atom. In the second `compute` command, a variable named `eatoms` is defined to store the sum of all `eng` values. This is just to show how to use computes with user-defined variables.  Notice that the `pe` energy column during minimization is equivalent to the `c_eatoms` column, as expected.

<br>
<div class="alert alert-block alert-info">
# ---------- Run Minimization --------------------- <br> 
reset_timestep 0  <br>
fix 1 all box/relax iso 0.0 vmax 0.001 <br>
thermo 10  <br>
thermo_style custom step pe lx ly lz press c_eatoms  <br>
min_style cg  <br>
minimize 1e-25 1e-25 5000 10000   <br>
</div>

The `reset_timestep` does just that. The `fix` command uses the `box/relax` setting, whereby all directions (`iso`) are relaxed to 0.0 Pa pressure for all atoms (`all`). The `thermo` specifies the output during minimization. The `thermo_style` specifies what type of output is shown to screen. Here I have used a `custom` list of metrics, including the time`step`, the potential energy (`pe`), the length of the box in x, y, and z (`lx`, `ly`, `lz`, respectively), the `press`ure, and the computed variable `eatoms`. The `min_style` specifies that conjugate gradient will be used for minimization and the `minimize` starts the minimization, *i.e.*, the simulation cell boundaries are relaxed from the specified lattice constant (4 Angstroms) to the actual lattice constant (4.05 Angstroms). 

<br>
<div class="alert alert-block alert-info">
variable natoms equal "count(all)" <br>
variable teng equal "c_eatoms" <br>
variable length equal "lx" <br>
variable ecoh equal "v_teng/v_natoms" 
</div>

This section defines some variables, such as `natoms` for the number of atoms, `teng` for the total potential energy, `length` for the length of the simulation cell, and `ecoh` for the cohesive energy of Al.

<br>
<div class="alert alert-block alert-info">
print "Total energy (eV) = \${teng};" <br>
print "Number of atoms = \${natoms};" <br>
print "Lattice constant (Angstoms) = \${length};" <br>
print "Cohesive energy (eV) = \${ecoh};" <br>
print "All done!"
</div>

This section prints these values to screen and to the logfile. The `${}` is used to insert the variables defined earlier. 

***
## FAQs 

<br>
<div class="alert alert-danger">
<strong>Question 1</strong>: But I wanted a simulation cell with <110> directions.  How would I change this?
</div>

Aha!  Yes, it is relatively easy to start changing the directions in the x, y, and z directions.  For instance, if you change this line: <br>
`lattice fcc 4 orient x 1 0 0 orient y 0 1 0 orient z 0 0 1`

to this one: <br>
`lattice fcc 4 orient x 1 1 0 orient y -1 1 0 orient z 0 0 1`

then you get a simulation cell that is oriented in the $<110>$ directions in the x- and y- directions.  If you mess up on the math, i.e., try 110 and 110 for the x and y directions (without the '-1'), then LAMMPS will return: <br>
`ERROR: Lattice orient vectors are not orthogonal`

So, if you try the $<110>$-oriented single crystal cell, LAMMPS automatically scales the x- and y-direction periodic lengths to 5.6568542 Angstroms (i.e., $4\sqrt{2}$).  After minimization, the cohesive energy per atom is `-3.35999998818379` ev, th
    
<br>
<div class="alert alert-danger">
<strong>Question 2</strong>: I typed in the above line and nothing happened.
</div>

Make sure that you are in the same directory as the LAMMPS executable. Make sure that you are typing in the correct filename. 

<br>
<div class="alert alert-danger">
<strong>Question 3</strong>: I keep getting an error with the potential, i.e., potential file not found.
</div>

First, check that the potential file is in the directory that you are running out of. Although, you can insert paths, if you want to store the potentials in another directory. Second, make sure that the potential file name is the same as that given in the input script. For instance, Windows has the habit of saving text files (like the potential files) with .txt extensions. If you happen to be running from a Windows operating system, I would change the settings so that you can see the extensions of the files as well. 

***
## Tutorial Links

[Click here to open the next tutorial](LAMMPS-Tutorials-02.ipynb)